In [1]:
from sympy import sin, cos, symbols, diff, Matrix
from sympy import solve, simplify
from sympy.physics.mechanics import *
from sympy import Rational
import math
init_vprinting()

# System Description

## coordinate frames

In [2]:
Sw = ReferenceFrame('Sw')
Sb = ReferenceFrame('Sb')
Sptmp = ReferenceFrame('Sptmp')
Sp = ReferenceFrame('Sp')
Sm2 = ReferenceFrame('Sm2')
Sm3 = ReferenceFrame('Sm3')

### general

In [3]:
t, g, pi = symbols('t g pi')

### base

In [4]:
mb, lb, rw = symbols('m_b l_b r_w')
Ibxx, Ibyy, Ibzz = symbols('I_{bxx} I_{byy} I_{bzz}')
Ib = inertia(Sb, Ibxx, Ibyy, Ibzz)

x, y, alpha = dynamicsymbols('x y alpha')

### pendulum

In [5]:
mp, lp = symbols('m_p l_p')
#ms, mm, mp, lgp, lp, lm = symbols('m_s m_m m_p l_{gp} l_p l_m') # full description
Ipxx, Ipyy, Ipzz = symbols('I_{pxx} I_{pyy} I_{pzz}')
Ipxx = Rational(1,12)*mp*lp**2
Ip = inertia(Sp, Ipxx, Ipxx, 0)

phi, theta = dynamicsymbols('phi theta')
#xi, eta, zeta = dynamicsymbols('xi eta zeta')

### motor

In [6]:
f1, f2, f3 = dynamicsymbols('f1 f2 f3')

# Basic Calculation

## state

In [7]:
q = Matrix([x, y, alpha, phi, theta])
qd = q.diff(t)

## coordinates transformation

In [8]:
Sb.orient(Sw, 'Axis', [alpha, Sw.z])

Sptmp.orient(Sw, 'Axis', [phi, Sw.x]);
Sp.orient(Sptmp, 'Axis', [theta, Sptmp.y]);
#Sp.set_ang_vel(Sw, phi.diff(t)*cos(theta)*Sw.x + theta.diff(t)*Sw.y + sin(theta)*phi.diff(t)*Sw.z) # omega
#display(Sp.ang_vel_in(Sw))

## point

### world

In [9]:
Pwo = Point('Pwo')
Pwo.set_vel(Sw, 0)

### body

In [10]:
Pbo = Point('Pbo')
Pbo.set_pos(Pwo, x*Sw.x + y*Sw.y + 0*Sw.z)
Pbo.set_vel(Sw, Pbo.pos_from(Pwo).dt(Sw))

### pendulum

In [11]:
Ppo = Point('Ppo')
Ppo.set_pos(Pwo, x*Sw.x + y*Sw.y + Rational(1,2)*lp*Sp.z)
#Ppo.set_pos(Pwo, Rational(1,2)*lp*Sp.z) # only pendulum case: just for debug
Ppo.set_vel(Sw, Ppo.pos_from(Pwo).dt(Sw))

### motor

In [12]:
Sm1 = Sb
Pm1 = Point("Pm1")
Pm1.set_pos(Pbo, lb*Sm1.x + 0*Sm1.y + 0*Sm1.z)

Sm2.orient(Sb, 'Axis', [Rational(2, 3)*pi, Sb.z])
Pm2 = Point("Pm2")
Pm2.set_pos(Pbo, lb*Sm2.x + 0*Sm2.y + 0*Sm2.z)

Sm3.orient(Sb, 'Axis', [Rational(4, 3)*pi, Sb.z])
Pm3 = Point("Pm3")
Pm3.set_pos(Pbo, lb*Sm3.x + 0*Sm3.y + 0*Sm3.z)

## position and velocity

In [13]:
BodyB = RigidBody('BodyB', Pbo, Sb, mb, (Ib, Pbo))
BodyP = RigidBody('BodyP', Ppo, Sp, mp, (Ip, Ppo))
p_cog_pos = (Ppo.pos_from(Pwo).express(Sw)).simplify()
display(p_cog_pos)

BodyP.potential_energy = mp * g * Rational(1,2) * lp * cos(phi) * cos(theta);
#display(BodyB.kinetic_energy)

## force from wheels

In [14]:
fl1 = (Pm1, f1*Sm1.y)
fl2 = (Pm2, f2*Sm2.y)
fl3 = (Pm3, f3*Sm3.y)
fl = [fl1, fl2, fl3]

# Lagrangian

## equation of motion

In [15]:
LL = Lagrangian(Sw, BodyB, BodyP)
LM = LagrangesMethod(LL, q, forcelist = fl, frame = Sw)

eom = simplify(LM.form_lagranges_equations())
f = simplify(LM.rhs())
display(eom)
display(f)

⎡                                                   ⎛             2                    ⎞           
⎢                                                mₚ⋅⎝- lₚ⋅sin(θ)⋅θ̇  + lₚ⋅cos(θ)⋅θ̈ + 2⋅ẍ⎠        
⎢                                        m_b⋅ẍ + ─────────────────────────────────────── + f₁⋅sin(
⎢                                                                   2                              
⎢                                                                                                  
⎢           ⎛                                       2                     2                        
⎢        mₚ⋅⎝lₚ⋅sin(φ)⋅sin(θ)⋅θ̈ + lₚ⋅sin(φ)⋅cos(θ)⋅φ̇  + lₚ⋅sin(φ)⋅cos(θ)⋅θ̇  + 2⋅lₚ⋅sin(θ)⋅cos(φ)
⎢m_b⋅ÿ + ─────────────────────────────────────────────────────────────────────────────────────────
⎢                                                                   2                              
⎢                                                                                                  


⎡                                                                                                  
⎢                                                                                                  
⎢                                                                                                  
⎢                                                                                                  
⎢                                                                                                  
⎢                                                                                                  
⎢                                                                                                  
⎢                                                                                                  
⎢                                                                                                  
⎢                                                                                                  


In [16]:
#mlatex(eom)

## linearization

In [17]:
linearizer = LM.to_linearizer(q_ind=q, qd_ind=qd)
op_point = {x: 0, x.diff(t): 0, y: 0, y.diff(t): 0, alpha: 0, alpha.diff(t): 0,
            phi: 0, phi.diff(t): 0, theta: 0, theta.diff(t): 0,
            f1: 0, f2: 0, f3: 0}
A, B = linearizer.linearize(A_and_B = True, op_point = op_point)

display(simplify(A))
display(simplify(B))

⎡0  0  0         0                0         1  0  0  0  0⎤
⎢                                                        ⎥
⎢0  0  0         0                0         0  1  0  0  0⎥
⎢                                                        ⎥
⎢0  0  0         0                0         0  0  1  0  0⎥
⎢                                                        ⎥
⎢0  0  0         0                0         0  0  0  1  0⎥
⎢                                                        ⎥
⎢0  0  0         0                0         0  0  0  0  1⎥
⎢                                                        ⎥
⎢                               3⋅g⋅mₚ                   ⎥
⎢0  0  0         0           ───────────    0  0  0  0  0⎥
⎢                            -4⋅m_b - mₚ                 ⎥
⎢                                                        ⎥
⎢             3⋅g⋅mₚ                                     ⎥
⎢0  0  0    ──────────            0         0  0  0  0  0⎥
⎢           4⋅m_b + mₚ                                  

⎡       0                0                0       ⎤
⎢                                                 ⎥
⎢       0                0                0       ⎥
⎢                                                 ⎥
⎢       0                0                0       ⎥
⎢                                                 ⎥
⎢       0                0                0       ⎥
⎢                                                 ⎥
⎢       0                0                0       ⎥
⎢                                                 ⎥
⎢                         ⎛2⋅π⎞            ⎛4⋅π⎞  ⎥
⎢                    4⋅sin⎜───⎟       4⋅sin⎜───⎟  ⎥
⎢                         ⎝ 3 ⎠            ⎝ 3 ⎠  ⎥
⎢       0           ───────────      ───────────  ⎥
⎢                   -4⋅m_b - mₚ      -4⋅m_b - mₚ  ⎥
⎢                                                 ⎥
⎢                        ⎛2⋅π⎞            ⎛4⋅π⎞   ⎥
⎢                   4⋅cos⎜───⎟       4⋅cos⎜───⎟   ⎥
⎢      4                 ⎝ 3 ⎠            ⎝ 3 ⎠   ⎥
⎢  ─────────

In [18]:
# only pendulum case: just for debug
#q = Matrix([phi, theta])
#qd = q.diff(t)
#LL = Lagrangian(Sw, BodyP)
#LM = LagrangesMethod(LL, q, forcelist = fl, frame = Sw)

#eom = LM.form_lagranges_equations().simplify()

#linearizer = LM.to_linearizer(q_ind=q, qd_ind=qd)
#op_point = {x: 0, x.diff(t): 0, y: 0, y.diff(t): 0, alpha: 0, alpha.diff(t): 0,
#            phi: 0, phi.diff(t): 0, theta: 0, theta.diff(t): 0,
#            f1: 0, f2: 0, f3: 0}
#A, B = linearizer.linearize(A_and_B = True, op_point = op_point)

#display(simplify(A))
#display(simplify(B))

In [19]:
display(BodyB.kinetic_energy(Sw).simplify())
display(BodyB.potential_energy.simplify())

In [20]:
display(BodyP.kinetic_energy(Sw).simplify())
display(BodyP.potential_energy.simplify())

# Verification

## rotation matrices

In [21]:
rotx_phi = Matrix([[1, 0, 0], [0, cos(phi), -sin(phi)], [0, sin(phi), cos(phi)]])
roty_theta = Matrix([[cos(theta), 0, sin(theta)], [0, 1, 0], [-sin(theta), 0, cos(theta)]])
#display(rotx_phi)
#display(roty_theta)

rot_op = rotx_phi * roty_theta
display(rot_op)

⎡    cos(θ)        0         sin(θ)    ⎤
⎢                                      ⎥
⎢sin(φ)⋅sin(θ)   cos(φ)  -sin(φ)⋅cos(θ)⎥
⎢                                      ⎥
⎣-sin(θ)⋅cos(φ)  sin(φ)  cos(φ)⋅cos(θ) ⎦

## cog position of pendulum

In [22]:
vec_p = Matrix([[0], [0], [Rational(1, 2)*lp]])
xi, eta, zeta = rot_op * vec_p
#display(xi)
#display(eta)
#display(zeta)

## angular velocity of pendulum

In [23]:
inertia_mat = Matrix([[Ipxx, 0, 0], [0, Ipxx, 0], [0, 0, 0]])
display(inertia_mat)
omega_p = Matrix([[0], [theta.diff(t)], [0]]) + roty_theta.transpose() * Matrix([[phi.diff(t)], [0], [0]])
display(omega_p)

⎡  2              ⎤
⎢lₚ ⋅mₚ           ⎥
⎢──────    0     0⎥
⎢  12             ⎥
⎢                 ⎥
⎢          2      ⎥
⎢        lₚ ⋅mₚ   ⎥
⎢  0     ──────  0⎥
⎢          12     ⎥
⎢                 ⎥
⎣  0       0     0⎦

⎡cos(θ)⋅φ̇⎤
⎢        ⎥
⎢   θ̇    ⎥
⎢        ⎥
⎣sin(θ)⋅φ̇⎦

## comparison of kinetic energy from algebraic calculation and sympy rigidbody

In [24]:
Tpt = Rational(1, 2) * mp * ((x.diff(t) + xi.diff(t))**2 + (y.diff(t) + eta.diff(t))**2 + zeta.diff(t)**2)
Tpr = Rational(1, 2) * omega_p.transpose() * inertia_mat * omega_p

display(simplify(Tpt + Tpr[0, 0]))
display(BodyP.kinetic_energy(Sw).simplify())

display(simplify(Tpt + Tpr[0, 0]) - BodyP.kinetic_energy(Sw).simplify())